# Dataset-Aware Inference Notebook


## Setup and Imports


In [1]:
import os
import sys
import json
import numpy as np
import torch
import monai
import matplotlib.pyplot as plt
import pprint
from tqdm import tqdm
from typing import Dict, List, Tuple, Optional

# MONAI imports
from monai.data import CacheDataset, DataLoader

# Import custom modules
from breast_segmentation.transforms import Preprocess
from breast_segmentation.models import BreastSegmentationModel, BreastFusionModel
from breast_segmentation.metrics.losses import (
    CABFL, SurfaceLossBinary, AsymmetricUnifiedFocalLoss, 
    AsymmetricFocalLoss, AsymmetricFocalTverskyLoss
)
from breast_segmentation.inference import (
    test_dataset_aware_no_patches,
    test_dataset_aware_fusion,
    test_dataset_aware_ensemble
)
from breast_segmentation.utils import (
    get_patient_ids, get_image_label_files_patient_aware, reverse_transformations,
    visualize_predictions, visualize_volume_predictions
)
from breast_segmentation.data import custom_collate_no_patches, custom_collate
from breast_segmentation.metrics import filter_masses

pp = pprint.PrettyPrinter(indent=2, width=100)

# Register loss classes for checkpoint loading
sys.modules['__main__'].CABFL = CABFL
sys.modules['__main__'].SurfaceLossBinary = SurfaceLossBinary
sys.modules['__main__'].AsymmetricUnifiedFocalLoss = AsymmetricUnifiedFocalLoss
sys.modules['__main__'].AsymmetricFocalLoss = AsymmetricFocalLoss
sys.modules['__main__'].AsymmetricFocalTverskyLoss = AsymmetricFocalTverskyLoss

c:\Users\pabli\AppData\Local\pypoetry\Cache\virtualenvs\venus-nCPuPPcI-py3.9\lib\site-packages\ignite\handlers\checkpoint.py:17: DeprecationWarning: `TorchScript` support for functional optimizers is deprecated and will be removed in a future PyTorch release. Consider using the `torch.compile` optimizer instead.
  from torch.distributed.optim import ZeroRedundancyOptimizer


## Configuration


In [2]:
# Settings
BATCH_SIZE = 128
NUM_WORKERS = os.cpu_count()
SEED = 200

# Data paths
BREADM_DIR = "./BreaDM"
TEST_DIR = os.path.join(BREADM_DIR, "seg/")
dataset_base_path = TEST_DIR 
CHECKPOINTS_DIR = "./checkpoints/breadm-dataset"

# Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Set seeds
torch.manual_seed(SEED)
np.random.seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)


Using device: cuda


## Load Test Patient IDs


In [3]:
# Get test patient IDs
x_test = get_patient_ids(TEST_DIR, split='test')
print(f"Number of test patients: {len(x_test)}")
print(f"Test patient IDs: {x_test[:5]}...")  # Show first 5


Number of test patients: 50
Test patient IDs: ['BreaDM-Be-1801', 'BreaDM-Be-1803', 'BreaDM-Be-1804', 'BreaDM-Be-1810', 'BreaDM-Be-1813']...


## Prepare Datasets


In [4]:
# Pre-computed statistics (from training)
GLOBAL_MEAN = 10.217766761779785
GLOBAL_STD = 26.677101135253906
PATCHES_MEAN =  20.630817413330078
PATCHES_STD = 35.328887939453125

# Create patient-aware datasets
datasets = {}
for patient_id in tqdm(x_test, desc="Creating patient datasets"):
    # Get files for this patient
    image_files, label_files = get_image_label_files_patient_aware(
        dataset_base_path="BreaDM/seg", 
        split="test", 
        image_type="VIBRANT+C2", 
        patient_id=patient_id
    )
    
    data_dicts = [
        {"image": img, "label": lbl, "subtracted": img}
        for img, lbl in zip(image_files, label_files)
    ]
    
    # Define transforms following rebuttal notebook pattern
    test_transforms_no_thorax_sub = monai.transforms.Compose([
        monai.transforms.LoadImaged(keys=["image", "label"], 
                                   image_only=False, 
                                   reader=monai.data.PILReader(converter=lambda image: image.convert("L"))),
        monai.transforms.EnsureChannelFirstd(keys=["image", "label"]),
        monai.transforms.Rotate90d(keys=["image", "label"]),
        Preprocess(keys=None, 
                  mode='test',  
                  dataset="BREADM", 
                  subtracted_images_path_prefixes=("VIBRANT+C2", "SUB2"), 
                  subtrahend=GLOBAL_MEAN, 
                  divisor=GLOBAL_STD, 
                  get_patches=False,
                  get_boundaryloss=False)
    ])
    
    test_transforms_patches_sub = monai.transforms.Compose([
        monai.transforms.LoadImaged(keys=["image", "label"], 
                                   image_only=False, 
                                   reader=monai.data.PILReader(converter=lambda image: image.convert("L"))),
        monai.transforms.EnsureChannelFirstd(keys=["image", "label"]),
        monai.transforms.Rotate90d(keys=["image", "label"]),
        Preprocess(keys=None, 
                  mode='test',  
                  dataset="BREADM", 
                  subtracted_images_path_prefixes=("VIBRANT+C2", "SUB2"), 
                  subtrahend=PATCHES_MEAN, 
                  divisor=PATCHES_STD, 
                  get_patches=True,
                  get_boundaryloss=False)
    ])
    
    # No thorax dataset (whole images)
    no_thorax_test_ds = CacheDataset(
        data=data_dicts,
        transform=test_transforms_no_thorax_sub,
        cache_rate=1.0,
        num_workers=NUM_WORKERS
    )
    
    # Patches dataset  
    patches_test_ds = CacheDataset(
        data=data_dicts,
        transform=test_transforms_patches_sub,
        cache_rate=1.0,
        num_workers=NUM_WORKERS
    )
    
    datasets[patient_id] = {
        'no_thorax_sub_test_ds': no_thorax_test_ds,
        'patches_sub_test_ds': patches_test_ds
    }

print("Patient datasets created successfully!")

# Define data_transforms dictionary for visualization function
data_transforms = {
    'no_thorax_sub_test_ds': test_transforms_no_thorax_sub,
    'no_thorax_sub_thorax_test_ds': test_transforms_no_thorax_sub,  # Same transform
    'patches_sub_test_ds': test_transforms_patches_sub
}


Creating patient datasets: 100%|██████████| 50/50 [00:58<00:00,  1.17s/it]

Patient datasets created successfully!


## Model Checkpoint Paths


In [5]:
model_paths = {
    # VENUS fusion models
    'venus_tiny': f'{CHECKPOINTS_DIR}/venus-tiny-best.ckpt',
    
    # Baseline models
    'unetplusplus': f'{CHECKPOINTS_DIR}/unetplusplus_model.ckpt',
    'skinny': f'{CHECKPOINTS_DIR}/skinny_model.ckpt',
    'resnet50': f'{CHECKPOINTS_DIR}/resnet50.ckpt',
    'fcn': f'{CHECKPOINTS_DIR}/unetplusplus_model.ckpt',  
    'segnet': f'{CHECKPOINTS_DIR}/segnet_model_large.ckpt',
    'swin': f'{CHECKPOINTS_DIR}/swin_model.ckpt',
    
    # Patches models
    'resnet50_patches': f'{CHECKPOINTS_DIR}/resnet50-patches.ckpt',
    'resnet18_patches': f'{CHECKPOINTS_DIR}/resnet18-patches.ckpt',
    'unetplusplus_patches': f'{CHECKPOINTS_DIR}/unetplusplus-patches-cabfl.ckpt',
}

# Check which models are available
print("Available models:")
for name, path in model_paths.items():
    if os.path.exists(path):
        print(f"  ✓ {name}: {path}")
    else:
        print(f"  ✗ {name}: {path} (not found)")


Available models:
  ✓ venus_tiny: ./checkpoints/breadm-dataset/venus-tiny-best.ckpt
  ✓ unetplusplus: ./checkpoints/breadm-dataset/unetplusplus_model.ckpt
  ✓ skinny: ./checkpoints/breadm-dataset/skinny_model.ckpt
  ✓ resnet50: ./checkpoints/breadm-dataset/resnet50.ckpt
  ✓ fcn: ./checkpoints/breadm-dataset/unetplusplus_model.ckpt
  ✓ segnet: ./checkpoints/breadm-dataset/segnet_model_large.ckpt
  ✓ swin: ./checkpoints/breadm-dataset/swin_model.ckpt
  ✓ resnet50_patches: ./checkpoints/breadm-dataset/resnet50-patches.ckpt
  ✓ resnet18_patches: ./checkpoints/breadm-dataset/resnet18-patches.ckpt
  ✓ unetplusplus_patches: ./checkpoints/breadm-dataset/unetplusplus-patches-cabfl.ckpt


## Test VENUS Model


In [6]:
# Test 1: venus-tiny-best.ckpt
if os.path.exists(model_paths['venus_tiny']):
    print("Testing VENUS Tiny ...")
    scores_for_statistics_fusion_tiny = test_dataset_aware_fusion(
        model_path=model_paths['venus_tiny'],
        patient_ids=x_test,
        datasets=datasets,
        whole_dataset_key="no_thorax_sub_test_ds",
        patches_dataset_key="patches_sub_test_ds",
        use_simple_fusion=False,
        use_decoder_attention=True,
        strict=True,
        filter=False,
        subtracted=True,
        get_scores_for_statistics=False,
        get_only_masses=False,
        base_channels=16
    )

    if scores_for_statistics_fusion_tiny:
        print("\nVENUS Tiny:")
        pp.pprint(scores_for_statistics_fusion_tiny)
else:
    print("venus-tiny.ckpt not found.")
    scores_for_statistics_fusion_tiny = None

Testing VENUS Tiny ...


c:\Users\pabli\AppData\Local\pypoetry\Cache\virtualenvs\venus-nCPuPPcI-py3.9\lib\site-packages\lightning\pytorch\utilities\parsing.py:210: Attribute 't_loss' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['t_loss'])`.


BreaDM-Be-1801


100%|██████████| 7/7 [00:00<00:00,  9.87it/s]


BreaDM-Be-1803


...

BreaDM-Ma-2140


100%|██████████| 11/11 [00:00<00:00, 17.02it/s]


BreaDM-Ma-2141


100%|██████████| 5/5 [00:00<00:00, 17.47it/s]


MODEL CLASS MEAN IOU  0.87886894
MODEL CLASS STD IOU  0.08537499

MODEL CLASS MEAN DICE  0.92374074
MODEL CLASS STD DICE  0.07815897

MODEL DIOU 0.9856115107913669
MODEL DIOU STD  0.11908593782191977

MODEL IOU MASS VOLUME  0.7583257555961609
MODEL IOU MASS VOLUME STD  0.17047640681266785

MODEL IOU MASS VOLUME NO EMPTY  0.7583257555961609
MODEL IOU MASS VOLUME NO EMPTY STD  0.1702718734741211

MODEL IOU MASS VOLUME NO EMPTY OPTIMISTIC  0.7583257555961609
MODEL IOU MASS VOLUME NO EMPTY OPTIMISTIC STD  0.1702718734741211
MODEL DICE MASS VOLUME  0.8477756977081299
MODEL DICE MASS VOLUME STD  0.15608349442481995

MODEL DICE MASS VOLUME NO EMPTY  0.8477756977081299
MODEL DICE MASS VOLUME NO EMPTY STD  0.15608349442481995

MODEL DICE MASS VOLUME NO EMPTY OPTIMISTIC  0.8477756977081299
MODEL DICE MASS VOLUME NO EMPTY OPTIMISTIC STD  0.15608349442481995

MODEL MEAN ACCURACY NO EMPTY 0.8788689374923706
MODEL MEAN ACCURACY NO EMPTY STD 0.08559996

MODEL MEAN PRECISION NO EMPTY 0.931920319795608

## Test Baseline Models


In [ ]:
# Test baseline models (6 tests)
baseline_tests = [
    ('unetplusplus', 'UNet++', 'unetplusplus'),
    ('skinny', 'SkinnyNet', 'skinny') ,
    ('resnet50', 'ResNet50', 'resnet50'),
    ('fcn', 'FCN', 'unetplusplus'), 
    ('segnet', 'SegNet', 'segnet'),
    ('swin', 'Swin-UNETR', 'swin_unetr')
]

baseline_results = {}
for model_key, model_name, arch_name in baseline_tests:
    if os.path.exists(model_paths[model_key]):
        print(f"Testing {model_name} model...")
        result = test_dataset_aware_no_patches(
            model_path=model_paths[model_key],
            patient_ids=x_test,
            datasets=datasets,
            dataset_key="no_thorax_sub_test_ds",
            filter=False,
            get_scores_for_statistics=False,
            get_only_masses=False,
            arch_name=arch_name,
            strict=True,
            subtracted=True
        )
        baseline_results[model_key] = result
        print(f"\n{model_name} Results:")

        if result:
            pp.pprint(result)
    else:
        print(f"{model_paths[model_key]} not found.")
        baseline_results[model_key] = None

print(f"\nCompleted {len([r for r in baseline_results.values() if r is not None])} baseline model tests.")


Testing UNet++ model...
BasicUNetPlusPlus features: (32, 32, 64, 128, 256, 32).
Initialized SurfaceLossBinary with [1]
BreaDM-Be-1801


100%|██████████| 7/7 [00:00<00:00, 25.13it/s]


BreaDM-Be-1803


100%|██████████| 3/3 [00:00<00:00, 26.31it/s]


BreaDM-Be-1804


...

BreaDM-Ma-2140


100%|██████████| 11/11 [00:00<00:00, 17.21it/s]


BreaDM-Ma-2141


100%|██████████| 5/5 [00:00<00:00, 18.64it/s]

MODEL CLASS MEAN IOU  0.86653
MODEL CLASS STD IOU  0.107399404

MODEL CLASS MEAN DICE  0.9101772
MODEL CLASS STD DICE  0.10570429

MODEL DIOU 0.9616306954436451
MODEL DIOU STD  0.19208618124221377

MODEL IOU MASS VOLUME  0.7336068749427795
MODEL IOU MASS VOLUME STD  0.2147994339466095

MODEL IOU MASS VOLUME NO EMPTY  0.7336068749427795
MODEL IOU MASS VOLUME NO EMPTY STD  0.2145417332649231

MODEL IOU MASS VOLUME NO EMPTY OPTIMISTIC  0.7336068749427795
MODEL IOU MASS VOLUME NO EMPTY OPTIMISTIC STD  0.2145417332649231
MODEL DICE MASS VOLUME  0.8206279873847961
MODEL DICE MASS VOLUME STD  0.21129119396209717

MODEL DICE MASS VOLUME NO EMPTY  0.8206279873847961
MODEL DICE MASS VOLUME NO EMPTY STD  0.21129119396209717

MODEL DICE MASS VOLUME NO EMPTY OPTIMISTIC  0.8206279873847961
MODEL DICE MASS VOLUME NO EMPTY OPTIMISTIC STD  0.21129119396209717

MODEL MEAN ACCURACY NO EMPTY 0.866530030965805
MODEL MEAN ACCURACY NO EMPTY STD 0.107566774

MODEL MEAN PRECISION NO EMPTY 0.9165849089622498
MO

## Test Ensemble Models


In [9]:

ensemble_tests = [
    ('venus_tiny', 'unetplusplus_patches', False, 16, 'VENUS Tiny + UNet++ patches'),
    ('venus_tiny', 'unetplusplus_patches', True, 16, 'VENUS Tiny + UNet++ patches (filtered)'),
    
    ('venus_tiny', 'resnet18_patches', False, 16, 'VENUS Tiny + ResNet18 patches'),
    ('venus_tiny', 'resnet18_patches', True, 16, 'VENUS Tiny + ResNet18 patches (filtered)'),
]

ensemble_results = {}
for whole_key, patches_key, use_filter, base_channels, description in ensemble_tests:
    if os.path.exists(model_paths[whole_key]) and os.path.exists(model_paths[patches_key]):
        print(f"Testing Ensemble: {description}...")
        
        result = test_dataset_aware_ensemble(
            model_whole_path=model_paths[whole_key],
            model_patches_path=model_paths[patches_key],
            patient_ids=x_test,
            datasets=datasets,
            whole_dataset_key="no_thorax_sub_test_ds",
            patches_dataset_key="patches_sub_test_ds",
            filter=use_filter,
            get_scores_for_statistics=False,
            get_only_masses=False,
            subtracted=True,
            base_channels=base_channels,
            strict = False
        )
        
        ensemble_key = f"{whole_key}+{patches_key}{'_filtered' if use_filter else ''}"
        ensemble_results[ensemble_key] = result
        print(f"\n{description} Results:")
        pp.pprint(result)
    else:
        print(f"Required models not found for ensemble: {description}")
        ensemble_key = f"{whole_key}+{patches_key}{'_filtered' if use_filter else ''}"
        ensemble_results[ensemble_key] = None

print(f"\nCompleted {len([r for r in ensemble_results.values() if r is not None])} ensemble model tests.")


c:\Users\pabli\AppData\Local\pypoetry\Cache\virtualenvs\venus-nCPuPPcI-py3.9\lib\site-packages\lightning\pytorch\utilities\parsing.py:210: Attribute 't_loss' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['t_loss'])`.


Testing Ensemble: VENUS Tiny + UNet++ patches...
BasicUNetPlusPlus features: (32, 32, 64, 128, 256, 32).
Initialized SurfaceLossBinary with [1]
BreastSegmentationModel(
  (model): BasicUNetPlusPlus(
    (conv_0_0): TwoConv(
      (conv_0): Convolution(
        (conv): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (adn): ADN(
          (N): InstanceNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=False)
          (D): Dropout(p=0.0, inplace=False)
          (A): LeakyReLU(negative_slope=0.1, inplace=True)
        )
      )
      (conv_1): Convolution(
        (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (adn): ADN(
          (N): InstanceNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=False)
          (D): Dropout(p=0.0, inplace=False)
          (A): LeakyReLU(negative_slope=0.1, inplace=True)
        )
      )
    )
    (conv_1_0): Down(
      (max_pooling): MaxPool2d(kernel_size=2,

100%|██████████| 7/7 [00:01<00:00,  6.40it/s]


BreaDM-Be-1803


100%|██████████| 3/3 [00:00<00:00,  8.30it/s]


...

filtering
num features: 3
fine
BreaDM-Ma-2140


100%|██████████| 11/11 [00:00<00:00, 11.78it/s]


filtering
num features: 5
fine
BreaDM-Ma-2141


100%|██████████| 5/5 [00:00<00:00, 11.18it/s]

filtering
num features: 1
fine
MODEL CLASS MEAN IOU  0.85927695
MODEL CLASS STD IOU  0.14805521

MODEL CLASS MEAN DICE  0.8921495
MODEL CLASS STD DICE  0.15310338

MODEL DIOU 0.882494004796163
MODEL DIOU STD  0.32202226055816824

MODEL IOU MASS VOLUME  0.7192621231079102
MODEL IOU MASS VOLUME STD  0.2957112193107605

MODEL IOU MASS VOLUME NO EMPTY  0.7192621231079102
MODEL IOU MASS VOLUME NO EMPTY STD  0.2953564524650574

MODEL IOU MASS VOLUME NO EMPTY OPTIMISTIC  0.7192621231079102
MODEL IOU MASS VOLUME NO EMPTY OPTIMISTIC STD  0.2953564524650574
MODEL DICE MASS VOLUME  0.7846537232398987
MODEL DICE MASS VOLUME STD  0.3058459162712097

MODEL DICE MASS VOLUME NO EMPTY  0.7846537232398987
MODEL DICE MASS VOLUME NO EMPTY STD  0.3058459162712097

MODEL DICE MASS VOLUME NO EMPTY OPTIMISTIC  0.7846537232398987
MODEL DICE MASS VOLUME NO EMPTY OPTIMISTIC STD  0.3058459162712097

MODEL MEAN ACCURACY NO EMPTY 0.8592770397663116
MODEL MEAN ACCURACY NO EMPTY STD 0.14840312

MODEL MEAN PRECISION N